In [1]:
import os
import subprocess

In [13]:
input_dir = r'R:\Aidan\test'
output_dir = r'R:\Aidan\test_output'
os.makedirs(output_dir, exist_ok=True)
im_list = ['B2_Region1']

In [12]:
rename =  [os.rename(os.path.join(input_dir, filename), os.path.join(input_dir, filename.replace('B2 Region1', 'B2_Region1'))) for filename in os.listdir(input_dir) if 'B2 Region1' in filename]

In [27]:
for label in im_list:
    
    # Construct the file series arguments for all rounds
    fileseries_args = [f"fileseries|{input_dir}|pattern={label}_ch_{{channel:2}}_m_{{series:2}}_t_{str(round_num).zfill(3)}.tif|overlap=0.105|width=5|height=5|layout=snake|pixel_size=0.325" for round_num in range(11)]

    # Create the full command as a list of arguments
    command = [
        'ashlar',
        *fileseries_args,
        '--output', f"{output_dir}\\{label}_ashlar_{{cycle:3}}_{{channel:2}}.tif",
        '--maximum-shift', '200',
        '--align-channel', '0'
    ]

    # Execute the command using subprocess
    subprocess.run(command)

In [29]:
command

['ashlar',
 'fileseries|R:\\Aidan\\test|pattern=B2_Region1_ch_{channel:2}_m_{series:2}_t_000.tif|overlap=0.105|width=5|height=5|layout=snake|pixel_size=0.325',
 'fileseries|R:\\Aidan\\test|pattern=B2_Region1_ch_{channel:2}_m_{series:2}_t_001.tif|overlap=0.105|width=5|height=5|layout=snake|pixel_size=0.325',
 'fileseries|R:\\Aidan\\test|pattern=B2_Region1_ch_{channel:2}_m_{series:2}_t_002.tif|overlap=0.105|width=5|height=5|layout=snake|pixel_size=0.325',
 'fileseries|R:\\Aidan\\test|pattern=B2_Region1_ch_{channel:2}_m_{series:2}_t_003.tif|overlap=0.105|width=5|height=5|layout=snake|pixel_size=0.325',
 'fileseries|R:\\Aidan\\test|pattern=B2_Region1_ch_{channel:2}_m_{series:2}_t_004.tif|overlap=0.105|width=5|height=5|layout=snake|pixel_size=0.325',
 'fileseries|R:\\Aidan\\test|pattern=B2_Region1_ch_{channel:2}_m_{series:2}_t_005.tif|overlap=0.105|width=5|height=5|layout=snake|pixel_size=0.325',
 'fileseries|R:\\Aidan\\test|pattern=B2_Region1_ch_{channel:2}_m_{series:2}_t_006.tif|overlap=0

In [8]:
im_list

[]